## Example for Kuramoto-Sivashinsky equation (KSE)

**CAUTION**: The following code using `ArtistAnimation` uses your network heavily.

In [ ]:
! cargo run --release --example kse > kse.csv

In [ ]:
%matplotlib nbagg

In [ ]:
import pandas as pd
import matplotlib.animation as animation
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("kse.csv", header=None).set_index(0)

In [ ]:
fig = plt.figure()

ims = []
for time, field in df.iterrows():
    im = plt.plot(field, color="blue")
    ims.append(im)
    
ani = animation.ArtistAnimation(fig, ims)
plt.show()